In [5]:
import numpy as np
from tqdm import tqdm
import logging

logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

import pandas as pd

################################################ data dimensions
_num_tuples = 15552 
_num_features = 45
_label_dims = 11

################################################ load data
data = pd.read_excel('spectrum_more_sets.xlsx').values
labels = pd.read_excel('temperature_more_sets.xlsx').values

#data = pd.read_excel('spectrum_norm.xlsx').values
#labels = pd.read_excel('temperature_norm.xlsx').values


################################################ Define dataset function
class Dataset(object):
    def __init__(self, data):
        # m, n denote number of tuples and features respectively
        self._m = data[0].shape[0]
        self._n = data[0].shape[1]
        self._training_data = data[0]
        self._training_labels = data[1]
        
    def __len__(self):
        return self._m
    
    def __getitem__(self, idx):
        return  self._training_data[idx,:], self._training_labels[idx,:] 

    def fetch_col(self, col_index):
        return self._training_data[:, col_index]

    def shuffle(self, seed=None):
        if seed:
            np.random.seed(seed=seed)
        shuffled_indices = np.arange(self._m)
        np.random.shuffle(shuffled_indices)
        self._training_data = np.take(self._training_data, shuffled_indices, axis=0)
        self._training_labels = np.take(self._training_labels, shuffled_indices)

    @property
    def num_tuples(self):
        return self._m

    @property
    def num_features(self):
        return self._n

    @property
    def labels(self):
        return self._training_labels

    @property
    def data_table(self):
        return self._training_data
       
dataset = Dataset((data, labels))

################################################ Define model

import torch
from torch import nn

# model definition
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(45, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 11)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.leakyrelu = nn.LeakyReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        #x = self.sigmoid(x)
        return x
    def name(self):
        return 'nn'

from torch.utils.data import DataLoader
import torch.nn.functional as F

args = {'lr':1e-3, 
        'max_steps':100,
        'batch_size':100,
        'epoch':10,
        'enable_gpu':None}
train_loader = DataLoader(dataset, batch_size=args['batch_size'], shuffle=True)

import torch.optim as optim

device = torch.device("cpu")
model = NN().to(device)
optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=0.9)

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in tqdm(enumerate(train_loader)):
        data, target = data.float().to(device), target.float().to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.mse_loss(output, target)
        loss.backward()
        optimizer.step()
        logger.info('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.mse_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    logger.info('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

################################################ Run the training set 
for epoch in range(1, args['epoch'] + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    #test(args, model, device, test_loader)

0it [00:00, ?it/s]INFO:root:Train Epoch: 1 [0/1295 (0%)]	Loss: 406713.156250
INFO:root:Train Epoch: 1 [100/1295 (8%)]	Loss: 387606.125000
INFO:root:Train Epoch: 1 [200/1295 (15%)]	Loss: 392749.843750
INFO:root:Train Epoch: 1 [300/1295 (23%)]	Loss: 396660.593750
INFO:root:Train Epoch: 1 [400/1295 (31%)]	Loss: 401099.312500
INFO:root:Train Epoch: 1 [500/1295 (38%)]	Loss: 339711.125000
INFO:root:Train Epoch: 1 [600/1295 (46%)]	Loss: 393590.875000
INFO:root:Train Epoch: 1 [700/1295 (54%)]	Loss: 521200.062500
INFO:root:Train Epoch: 1 [800/1295 (62%)]	Loss: 409492.406250
INFO:root:Train Epoch: 1 [900/1295 (69%)]	Loss: 406882.468750
INFO:root:Train Epoch: 1 [1000/1295 (77%)]	Loss: 400323.468750
INFO:root:Train Epoch: 1 [1100/1295 (85%)]	Loss: 400491.093750
INFO:root:Train Epoch: 1 [1140/1295 (92%)]	Loss: 400060.000000
13it [00:00, 147.71it/s]
0it [00:00, ?it/s]INFO:root:Train Epoch: 2 [0/1295 (0%)]	Loss: 402170.500000
INFO:root:Train Epoch: 2 [100/1295 (8%)]	Loss: 389156.187500
INFO:root:Trai

INFO:root:Train Epoch: 10 [1100/1295 (85%)]	Loss: 3398.846680
INFO:root:Train Epoch: 10 [1140/1295 (92%)]	Loss: 3644.958252
13it [00:00, 158.49it/s]
